## 1. Test LLM and prompt types

In [ ]:
from configuration.llm_inference_configuration import APILLMConfiguration
from llm.azure_openai import AzureOpenAILLM
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 2. Test tools

### 2.1. Python code runner tool

In [1]:
from tools.python_code_runner import PythonCodeRunnerTool
from configuration.tool_configuration import ToolConfiguration

In [2]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [4]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = python_code_runner.execute(input_code_list=input_code_list)
results

[{'id': '1',
  'result': {'a': 11, 'b': 20, 'result': 31, 'message': 'Hello, World!'}},
 {'id': '2', 'result': {'num1': 5, 'num2': 3, 'result': 15}}]

In [2]:
from base_classes.memory.management_term import MemoryState


isinstance(MemoryState.FREE, MemoryState)

True